In [21]:
%matplotlib inline
import matplotlib.pyplot as plt

# Logistic Regression on imdb for sentiment

https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

In [ ]:
import urllib.request,tarfile
import os,tempfile
os.chdir(tempfile.gettempdir())
urllib.request.urlretrieve('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz','aclImdb_v1.tar.gz')
tarfile.open('aclImdb_v1.tar.gz','r:gz').extractall()
os.chdir('aclImdb')
cmd='for split in train test; do for sentiment in pos neg; do for file in $split/$sentiment/*; do cat $file >> full_${split}.txt; echo >> full_${split}.txt; done; done; done;'
os.sysmtem(cmd)

In [24]:
import nltk, string
wnl = nltk.WordNetLemmatizer()
reviews_train=[' '.join(wnl.lemmatize(w) for w in nltk.word_tokenize(l) if w.isalpha())
               for l in open('full_train.txt')]
reviews_test=[' '.join(wnl.lemmatize(w) for w in nltk.word_tokenize(l) if w.isalpha())
              for l in open('full_test.txt')]

## BOW - 1-gram

features = 1/0 * vocab size (exist/not exist)

target = 1/0 (positive/negative)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train)
X = cv.transform(reviews_train)
y = [1 if i < 12500 else 0 for i in range(25000)]

X_test = cv.transform(reviews_test)

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

for c in [0.01, 0.03, 0.1, 0.3, 1]:  
    print("Accuracy for C={}: {}".format(c,
                                         np.mean(cross_val_score(LogisticRegression(C=c,solver='liblinear'), X,y, cv=5, scoring='accuracy'))
                                        )
         )

Accuracy for C=0.01: 0.8612399999999999
Accuracy for C=0.03: 0.86492
Accuracy for C=0.1: 0.8626000000000001
Accuracy for C=0.3: 0.85856
Accuracy for C=1: 0.8500400000000001


In [53]:
from sklearn.metrics import accuracy_score
model=LogisticRegression(C=0.03).fit(X,y)
print ("Final Accuracy: %s" % accuracy_score(target, model.predict(X_test)))
feature_to_coef = {
    word: coef for word, coef in zip( cv.get_feature_names(), model.coef_[0] )
}
for best_positive in sorted( feature_to_coef.items(),  key=lambda x: x[1],  reverse=True)[:5]:
    print (best_positive)    
for best_negative in sorted( feature_to_coef.items(),  key=lambda x: x[1])[:5]:
    print (best_negative)

Final Accuracy: 0.87772
('excellent', 0.8593489932761398)
('perfect', 0.6801029769414313)
('great', 0.6545590135908305)
('amazing', 0.5924591149829764)
('favorite', 0.5651963748851384)
('worst', -1.2600324008053674)
('waste', -1.0270444543800354)
('awful', -0.8978979413808582)
('boring', -0.7401303362319944)
('poorly', -0.717995970049884)


## 2-gram?

# Word2Vec

https://github.com/udacity/deep-learning-v2-pytorch/tree/master/word2vec-embeddings

## Data

In [4]:
url='https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip'
import urllib.request,zipfile
import os,tempfile
os.chdir(tempfile.gettempdir())
urllib.request.urlretrieve(url,'text.zip')
zipfile.ZipFile('text.zip').extractall()
with open('text8') as f:
    text = f.read()
print(text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


### Preprocess

In [5]:
# Tokenize
import nltk
from collections import Counter
from string import punctuation
for p in punctuation:
    text=text.replace(p,f' {p} ')

words = text.split()
cnt = Counter(words)
words=[w for w in words if cnt[w]>5]
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [6]:
# Lookup table with descending frequency
cnt = Counter(words)
sorted_vocab = sorted(cnt, key=cnt.get, reverse=True)
int2vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
vocab2int = {word: ii for ii, word in int2vocab.items()}
integers = [vocab2int[w] for w in words]
Nw = len(words)
Nv = len(cnt)
print(f"{Nw} words, {Nv} vocabs")

16680599 words, 63641 vocabs


**Mikolov subsampling**: Discard $i^{\rm th}$ word with frequency $$P_\text{discard}(w_i)=1-\sqrt{\frac{\rm thres}{\text{total number of $w_i$ / total  number words}}},$$
So that frequent words `the`, `of`, etc. are downsampled.

In [7]:
# Mikolov subsampling
import random, numpy as np
thres = 1e-5
cnt =  Counter(integers)
P_dis = {i:1-np.sqrt(thres/(n/Nw)) for i,n in cnt.items()}
train = [i for i in integers if random.random()>P_dis[i]]
print(train[:10])

[5233, 3133, 10571, 27349, 15067, 58112, 10712, 1324, 454, 2731]


### Batch Loader

Advantage of random window size: closer words are included more frequently than distant words => higher importance to close words

In [8]:
import numpy as np
def get_surround(li, idx, minsize=1, maxsize=5): # list of neighboring elements randomly of size 2*(minsize,maxsize)
    R = np.random.randint(minsize, maxsize+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    win = li[start:idx] + li[idx+1:stop+1]    
    return list(win)
get_surround(list(range(10)), idx=5, minsize=3, maxsize=3)

[2, 3, 4, 6, 7, 8]

In [9]:
def get_batches(words, batch_size, maxsize=5): #generates (list of centre, list of surround), with duplicated element in c
    n_batches = len(words)//batch_size
    words = words[:n_batches*batch_size]    
    for idx in range(0, len(words), batch_size):
        c, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_c = batch[ii]
            batch_y = get_surround(batch, ii, maxsize)
            y.extend(batch_y)
            c.extend([batch_c]*len(batch_y))
        yield c, y
        
next(get_batches(list(range(10)), batch_size=4))

([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], [1, 2, 3, 0, 2, 3, 0, 1, 3, 0, 1, 2])

## SkipGram using Torch

In [10]:
import torch
cuda=torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
torch.__version__, cuda

('1.1.0', False)

In [11]:
embedding_dim=300
print_every = 500
steps = 0
epochs = 5

### Model

SkipGram: predict surrounding words from centre word
* `nn.Embedding`: converts word to vec
* `nn.Linear`: dense embed_dim $\rightarrow$ n_vocab (the prediction)
* `nn.LogSoftmax`: converts to probability

In [12]:
from torch import nn
import torch.optim as optim

class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()        
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.embed(x)
        scores = self.output(x)
        log_ps = self.log_softmax(scores)        
        return log_ps
    
model = SkipGram(Nv, embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

### Validation function

During training, want some words examples that the model considers "similar"

In [13]:
def similar_examples(embedding, sample_size=10, topk=5, device='cpu'):
    """ Returns list of exampleidx, list of list of idx similar to the corresponding exampleidx """
        
    embed_vectors = embedding.weight  # shape (Nv, embedding_dim)
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0) # = |b|
    
    # pick N vocab id from ranges (0,100) and (1000,1100)  (corr. frequent & infrequent words)
    valid_examples = np.array(random.sample(range(100), sample_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1100), sample_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples) # shape (sample_size, embedding_dim)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes # sim=(a.b)/|a||b|  (common denom |a| discarded)
    
    _, closest_idxs = similarities.topk(topk+1) # highest similarities (includes itself)
    
    return valid_examples.to('cpu'), closest_idxs[:,1:].to('cpu') #exclude itself

### Train

In [ ]:
step = 0
for _ in range(epochs):
    
    for centre, surround in get_batches(train, 256):
        step+=1
        inputs  = torch.LongTensor(centre).to(device)
        targets = torch.LongTensor(surround).to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step%print_every: continue
        # [OPTIONAL] getting examples and similarities
        from IPython.display import clear_output
        clear_output()
        for exampleidx, closeidxs in zip(*similar_examples(model.embed, device=device)):
            print(int2vocab[exampleidx.item()] + " ~ " + ', '.join(int2vocab[idx.item()] for idx in closeidxs))

## SkipGram + Negative Sampling

https://github.com/udacity/deep-learning-v2-pytorch/blob/master/word2vec-embeddings/Negative_Sampling_Solution.ipynb

## Visualization

In [ ]:
from sklearn.manifold import TSNE
embeddings = model.embed.weight.to('cpu').data.numpy()
viz_words = 60
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int2vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)

# Word Embedding + sentiment of imdb

https://www.tensorflow.org/alpha/tutorials/text/word_embeddings

In [1]:
import tensorflow as tf
from tensorflow import keras

## Load Data

In [10]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=vocab_size) # discard infrequent words
# https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[1][:20])

(25000,) (25000,) (25000,) (25000,)
[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14]


In [11]:
# int2words function
word_index = keras.datasets.imdb.get_word_index()
# reserve first 4 indices
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(X_train[1])

"<START> big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal <UNK> the hair is big lots of boobs <UNK> men wear those cut <UNK> shirts that show off their <UNK> sickening that men actually wore them and the music is just <UNK> trash that plays over and over again in almost every scene there is trashy music boobs and <UNK> taking away bodies and the gym still doesn't close for <UNK> all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

In [12]:
# pad/truncate the same length
maxlen = 500
X_train = keras.preprocessing.sequence.pad_sequences(X_train,
                                                     value=word_index["<PAD>"],
                                                     padding='post',
                                                     maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test,
                                                    value=word_index["<PAD>"],
                                                    padding='post',
                                                    maxlen=maxlen)
set(map(len,X_train))|set(map(len,X_test))

{500}

## Model

In [18]:
embedding_dim=16

model = keras.Sequential([
  keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(16, activation='relu'),
  keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Train

In [20]:
import tempfile
_,weightsfile=tempfile.mkstemp()
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
es=EarlyStopping(monitor='val_loss', patience=5, verbose=1)
mc=ModelCheckpoint(weightsfile,monitor='val_loss',save_best_only=True,save_weights_only=True,verbose=0)

history = model.fit(X_train, y_train, epochs=300, batch_size=64,
                    callbacks=[es,mc],
                    validation_split=0.2)
model.load_weights(weightsfile)

Train on 20000 samples, validate on 5000 samples
Epoch 1/300
20000/20000 [==============================] - 1s 69us/sample - loss: 0.6692 - accuracy: 0.6573 - val_loss: 0.5949 - val_accuracy: 0.7776
Epoch 2/300
20000/20000 [==============================] - 1s 58us/sample - loss: 0.4666 - accuracy: 0.8402 - val_loss: 0.3800 - val_accuracy: 0.8662
Epoch 3/300
20000/20000 [==============================] - 1s 62us/sample - loss: 0.3168 - accuracy: 0.8841 - val_loss: 0.3168 - val_accuracy: 0.8794: 0.3280 
Epoch 4/300
20000/20000 [==============================] - 1s 60us/sample - loss: 0.2587 - accuracy: 0.9040 - val_loss: 0.2962 - val_accuracy: 0.8828
Epoch 5/300
20000/20000 [==============================] - 1s 61us/sample - loss: 0.2233 - accuracy: 0.9192 - val_loss: 0.2818 - val_accuracy: 0.8902
Epoch 6/300
20000/20000 [==============================] - 1s 59us/sample - loss: 0.1989 - accuracy: 0.9284 - val_loss: 0.2775 - val_accuracy: 0.8920
Epoch 7/300
20000/20000 [=================

## Evaluation

### Training and Validation loss history

In [ ]:
history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

### Retrieve learned embeddings

In [24]:
weights = model.layers[0].get_weights()[0]          # embedding layer weights
print(weights.shape)                                # vocab size -> embedding dimension

(10000, 16)


### Test data

In [25]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 0s 19us/sample - loss: 0.2825 - accuracy: 0.8866


[0.28253252049446104, 0.88656]

### Sample/new data

In [58]:
print(model(X_test[:3]).numpy())
for i in range(3):
    print(decode_review(X_test[i]).strip('<PAD>STR '))

[[0.17863826]
 [0.99984455]
 [0.8195051 ]]
please give this one a miss br br <UNK> <UNK> and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite <UNK> so all you madison fans give this a miss
this film requires a lot of patience because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in frances <UNK> the sandy dennis character apartment but the film builds to a disturbing climax br br the characters create an atmosphere <UNK> with sexual tension and psychological <UNK> it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it

In [57]:
import numpy as np
sent='used to like his movies this is an exception'
X=[word_index[w] for w in sent.split()]
X+=[word_index["<PAD>"]]*(maxlen-len(X))
model(np.array([X])).numpy()

array([[0.51735705]], dtype=float32)

# Classification by CNN

compared with n-gram (counting, then computing MLE).

Now representing words by vec, use CNN along time for look for pattern (larger size = longer-grams).

https://www.kaggle.com/babakgohardani/spam-detection-with-deep-learning-methods

# Language modelling by MLP